In [87]:
import gym
import tensorflow as tf
import numpy as np
np.set_printoptions(precision=3)
import random
import datetime
import itertools

In [353]:
def weight_variable(shape, name=None):
  initial = tf.truncated_normal(shape, mean=0.0, stddev=0.1)
  return tf.Variable(initial, name=name)

def bias_variable(shape, name=None):
  initial = tf.constant(0.0, shape=shape)
  return tf.Variable(initial, name=name)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [89]:
sess = tf.InteractiveSession()

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x107ec6c10>> ignored


In [90]:
 env = gym.make('CartPole-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('TwoRoundNondeterministicReward-v0')

print(env.action_space)
print(env.observation_space)

[2016-05-27 23:31:22,847] Making new env: CartPole-v0


Discrete(2)
Box(4,)


In [91]:
def one_hot(index, name="one_hot"):
    return tf.one_hot(indices=index, depth=env.observation_space.n, on_value=1, off_value=0, axis=None, name=name).eval()

In [92]:
def matvecmul(mat, vec):
    return tf.reshape(tf.matmul(tf.reshape(vec, [1, -1]), mat), [-1])

In [93]:
def getFeedDict(observation):
    if type(env.observation_space) == gym.spaces.discrete.Discrete:
        return {x:[one_hot(observation)]}
    elif type(env.observation_space) == gym.spaces.box.Box:
        return {x:[observation]}
    else:
        print "ERR"

In [94]:
if type(env.observation_space) == gym.spaces.discrete.Discrete:
    inputDims = env.observation_space.n
elif type(env.observation_space) == gym.spaces.box.Box:
    inputDims = env.observation_space.shape[0]
else:
    print "ERR"
print "inputDims: ", inputDims
    
if type(env.action_space) == gym.spaces.discrete.Discrete:
    outputDims = env.action_space.n
elif type(env.action_space) == gym.spaces.box.Box:
    outputDims = env.action_space.shape[0]
else:
    print "ERR"
print "outputDims: ", outputDims

x = tf.placeholder(tf.float32, [None, inputDims], name="x")
W = weight_variable([inputDims, outputDims], name="W")
b = bias_variable([outputDims], name="b")
q = tf.matmul(x, W) + b

greedyAction = tf.argmax(q, 1, name="greedyAction")
greedyValue = tf.reduce_max(q, name="greedyValue")
greedyActionValue = tf.gather(tf.gather(q, 0), greedyAction, name="greedyActionValue")
        
sess.run(tf.initialize_all_variables())

inputDims:  4
outputDims:  2


In [95]:
oldObservations = []
actions = []
observations = []
rewards = []
dones = []

for i_episode in range(30):
    observation = env.reset()
    
    for timestep in itertools.count():
        env.render()
        
        epsilon = 1.0
        if random.random() < epsilon:
            action = env.action_space.sample() #choose a random action
        else:
            action = greedyAction.eval(getFeedDict(observation))
        
        oldObservation = observation
        
        observation, reward, done, info = env.step(action)
        
        oldObservations.append(oldObservation)
        actions.append(action)
        observations.append(observation)
        rewards.append(reward)
        dones.append(done)
        
        if done:
            print("Episode finished after {} timesteps".format(timestep+1))
            break

Episode finished after 18 timesteps
Episode finished after 13 timesteps
Episode finished after 23 timesteps
Episode finished after 21 timesteps
Episode finished after 30 timesteps
Episode finished after 24 timesteps
Episode finished after 36 timesteps
Episode finished after 19 timesteps
Episode finished after 11 timesteps
Episode finished after 16 timesteps
Episode finished after 11 timesteps
Episode finished after 16 timesteps
Episode finished after 31 timesteps
Episode finished after 24 timesteps
Episode finished after 19 timesteps
Episode finished after 16 timesteps
Episode finished after 21 timesteps
Episode finished after 15 timesteps
Episode finished after 19 timesteps
Episode finished after 28 timesteps
Episode finished after 89 timesteps
Episode finished after 34 timesteps
Episode finished after 27 timesteps
Episode finished after 14 timesteps
Episode finished after 40 timesteps
Episode finished after 17 timesteps
Episode finished after 36 timesteps
Episode finished after 15 ti

In [71]:
if done:
    td_target = reward
else:
    td_target = greedyValue.eval(getFeedDict(observation)) + reward
#         print "td_target: ", td_target

td_error = tf.square(tf.sub(np.array(td_target, dtype=np.float32), greedyActionValue), name="td_error")
#         print getFeedDict(oldObservation)
#         print(td_error)
#         print(action_value.eval(getFeedDict(oldObservation)))
#         print "td_error: ", td_error.eval(getFeedDict(oldObservation))


train_step = tf.train.GradientDescentOptimizer(0.1).minimize(td_error)

sess.run(train_step, getFeedDict(oldObservation))

#         print("Now the network has been improved.")
#         print "action_value: ", action_value.eval(getFeedDict(oldObservation))
#         print 


#         if type(env.observation_space) == gym.spaces.discrete.Discrete:
#             for obs in xrange(env.observation_space.n):
#                 print "q("+str(obs)+"): ", q.eval(getFeedDict(obs))

before:  2016-05-27 23:21:56.172650
after:  2016-05-27 23:21:56.550911


In [349]:
envModelInsData = np.concatenate((np.asarray(oldObservations), np.asarray([actions]).T), 1)
envModelOutsData = np.concatenate((np.asarray(observations), np.asarray([dones]).T, np.asarray([rewards]).T), 1)

envModelIns = tf.placeholder(tf.float32, [None, envModelInsData.shape[1]], name="envModelIns")
envModelOuts = tf.placeholder(tf.float32, [None, envModelOutsData.shape[1]], name="envModelOuts")

envModelW = weight_variable([envModelInsData.shape[1], envModelOutsData.shape[1]], name="envModelW")
envModelb = bias_variable([envModelOutsData.shape[1]], name="envModelb")
envModelPredictedOuts = tf.matmul(envModelIns, envModelW) + envModelb

loss = tf.reduce_mean(tf.square(tf.sub(envModelPredictedOuts, envModelOuts)))

sess.run(tf.initialize_all_variables())

In [356]:
for i in xrange(20):
    miniBatchIndices = np.random.randint(envModelInsData.shape[0], size=100)
    
    tf.train.GradientDescentOptimizer(0.5).minimize(loss).run(feed_dict={
        envModelIns: envModelInsData[miniBatchIndices, :],
        envModelOuts: envModelOutsData[miniBatchIndices, :]
    })

In [357]:
print envModelW.eval()
print envModelb.eval()
print loss.eval(feed_dict={
    envModelIns: envModelInsData,
    envModelOuts: envModelOutsData
})

[[ 0.062 -0.121  0.045 -0.084 -0.075 -0.02 ]
 [ 0.018  0.369  0.022 -0.419 -0.08  -0.07 ]
 [-0.124 -0.028  0.021 -0.004  0.004 -0.028]
 [-0.045 -0.415  0.109  0.742 -0.02  -0.045]
 [ 0.005  0.293 -0.005 -0.408 -0.008  0.09 ]]
[ 0.01  -0.098  0.013  0.233  0.059  0.957]
0.0137582


In [355]:
print "envModelInsData[:3]: "
print envModelInsData[:3]
print
print "envModelPredictedOuts.eval(feed_dict={envModelIns: envModelInsData[:3]}): "
print envModelPredictedOuts.eval(feed_dict={envModelIns: envModelInsData[:3]})
print
print "envModelOutsData[:3]: "
print envModelOutsData[:3]
print 
print "envModelOutsData[:3] - envModelInsData[:3,:5]: "
print envModelOutsData[:3,:5] - envModelInsData[:3]
print 
print "envModelOutsData[:3] - envModelPredictedOuts.eval(feed_dict={envModelIns: envModelInsData[:3]}): "
print envModelOutsData[:3] - envModelPredictedOuts.eval(feed_dict={envModelIns: envModelInsData[:3]})

envModelInsData[:3]: 
[[-0.004  0.027 -0.023  0.038  1.   ]
 [-0.003  0.222 -0.022 -0.262  0.   ]
 [ 0.001  0.028 -0.028  0.024  0.   ]]

envModelPredictedOuts.eval(feed_dict={envModelIns: envModelInsData[:3]}): 
[[ 0.021  0.141  0.009 -0.067  0.022  1.07 ]
 [ 0.026  0.119 -0.013 -0.127  0.035  0.893]
 [ 0.01  -0.061  0.016  0.15   0.047  0.898]]

envModelOutsData[:3]: 
[[-0.003  0.222 -0.022 -0.262  0.     1.   ]
 [ 0.001  0.028 -0.028  0.024  0.     1.   ]
 [ 0.002 -0.167 -0.027  0.307  0.     1.   ]]

envModelOutsData[:3] - envModelInsData[:3,:5]: 
[[  5.398e-04   1.954e-01   7.565e-04  -2.999e-01  -1.000e+00]
 [  4.449e-03  -1.948e-01  -5.241e-03   2.856e-01   0.000e+00]
 [  5.528e-04  -1.947e-01   4.702e-04   2.839e-01   0.000e+00]]

envModelOutsData[:3] - envModelPredictedOuts.eval(feed_dict={envModelIns: envModelInsData[:3]}): 
[[-0.025  0.081 -0.031 -0.195 -0.022 -0.07 ]
 [-0.025 -0.092 -0.015  0.151 -0.035  0.107]
 [-0.008 -0.106 -0.043  0.157 -0.047  0.102]]


In [341]:
envModelb.assign(np.zeros((5))).eval()

array([ 0.,  0.,  0.,  0.,  0.], dtype=float32)